# This is a replication of a medium post
  
   [Link](https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from __future__ import print_function, division
import numpy as np
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
num_epoches = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

In [3]:
np.random.choice(2, total_series_length, p = [0.5, 0.5])

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
def generate_data():
    x = np.array(np.random.choice(2, total_series_length, p = [0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[:echo_step] = 0
    x = np.reshape(x, (batch_size, -1))
    y = np.reshape(y, (batch_size, -1))
    return x, y

In [5]:
batch_x = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batch_y = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])
init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [6]:
batch_x

<tf.Tensor 'Placeholder:0' shape=(5, 15) dtype=float32>

In [7]:
W = tf.Variable(np.random.rand(state_size+1, state_size), dtype=tf.float32)
b = tf.Variable(np.zeros((1,state_size)), dtype=tf.float32)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [8]:
inputs_series = tf.unstack(batch_x, axis=1)
labels_series = tf.unstack(batch_y, axis=1)

In [9]:
inputs_series

[<tf.Tensor 'unstack:0' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:1' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:2' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:3' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:4' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:5' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:6' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:7' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:8' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:9' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:10' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:11' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:12' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:13' shape=(5,) dtype=float32>,
 <tf.Tensor 'unstack:14' shape=(5,) dtype=float32>]

In [10]:
current_state = init_state
states_series = []
for current_input in inputs_series:
    current_input = tf.reshape(current_input, [batch_size, 1])
    input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # Increasing number of columns

    next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
    states_series.append(next_state)
    current_state = next_state

In [11]:
logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [12]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)


In [13]:
init_op = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []
    
    for epoche_id in range(num_epoches):
        x, y = generate_data()
        _current_state = np.zeros((batch_size, state_size))
        print("New data, epoch", epoche_id)
        
        for batch_idx in range(num_epoches):
            
            start_id = batch_idx * truncated_backprop_length
            end_id = start_id + truncated_backprop_length
            batchX = x[:, start_id:end_id]
            batchY = x[:, start_id:end_id]
            
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batch_x:batchX,
                    batch_y:batchY,
                    init_state:_current_state
                        })
            loss_list.append(_total_loss)
            if batch_idx % 100:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()                

New data, epoch 0
Step 1 Loss 0.669695


/usr/local/lib/python3.5/dist-packages/matplotlib/backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Step 2 Loss 0.729473
Step 3 Loss 0.744917
Step 4 Loss 0.649151
Step 5 Loss 0.583867
Step 6 Loss 0.50801
Step 7 Loss 0.45186
Step 8 Loss 0.434737
Step 9 Loss 0.351821
Step 10 Loss 0.299451
Step 11 Loss 0.260501
Step 12 Loss 0.231615
Step 13 Loss 0.20275
Step 14 Loss 0.172664
Step 15 Loss 0.16933
Step 16 Loss 0.150566
Step 17 Loss 0.130533
Step 18 Loss 0.123169
Step 19 Loss 0.117866
Step 20 Loss 0.104029
Step 21 Loss 0.0990989
Step 22 Loss 0.0898847
Step 23 Loss 0.0832005
Step 24 Loss 0.080446
Step 25 Loss 0.0732768
Step 26 Loss 0.0677279
Step 27 Loss 0.0657896
Step 28 Loss 0.0610773
Step 29 Loss 0.0575607
Step 30 Loss 0.0566514
Step 31 Loss 0.0512125
Step 32 Loss 0.0490658
Step 33 Loss 0.0472349
Step 34 Loss 0.044633
Step 35 Loss 0.0431001
Step 36 Loss 0.041111
Step 37 Loss 0.0388044
Step 38 Loss 0.0369858
Step 39 Loss 0.0363976
Step 40 Loss 0.0349202
Step 41 Loss 0.0334496
Step 42 Loss 0.031147
Step 43 Loss 0.0306806
Step 44 Loss 0.0305335
Step 45 Loss 0.0292634
Step 46 Loss 0.027812
S

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1553, in __call__
    return self.func(*args)
  File "/usr/lib/python3.5/tkinter/__init__.py", line 599, in callit
    func(*args)
  File "/usr/local/lib/python3.5/dist-packages/matplotlib/backends/backend_tkagg.py", line 370, in idle_draw
    self.draw()
  File "/usr/local/lib/python3.5/dist-packages/matplotlib/backends/backend_tkagg.py", line 351, in draw
    FigureCanvasAgg.draw(self)
  File "/usr/local/lib/python3.5/dist-packages/matplotlib/backends/backend_agg.py", line 464, in draw
    self.figure.draw(self.renderer)
  File "/usr/local/lib/python3.5/dist-packages/matplotlib/artist.py", line 63, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "/usr/local/lib/python3.5/dist-packages/matplotlib/figure.py", line 1143, in draw
    renderer, self, dsu, self.suppressComposite)
  File "/usr/local/lib/python3.5/dist-packages/matplotlib/image.py", line 1

Step 95 Loss 0.00085053
Step 96 Loss 0.00085374
Step 97 Loss 0.000846809
Step 98 Loss 0.000856358
Step 99 Loss 0.00084439
New data, epoch 7
Step 1 Loss 0.000819282
Step 2 Loss 0.000838244
Step 3 Loss 0.000844297
Step 4 Loss 0.000837172
Step 5 Loss 0.000854273
Step 6 Loss 0.000823135
